In [1]:
# Self contained memout assertion 

##########################################################################
#   IMPORT BLOCK                                                         #
##########################################################################
import gc 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import checkpoints

##########################################################################
#   FUNCTION BLOCK                                                       #
##########################################################################
def memout_example():    
    assert vars() == {}
    # empty slate 

    # build persistent data 
    val_loader = cifar_loader.load_cifar_data('val', normalize=False, batch_size=16, use_gpu=False)
    
    # now loop through batches and show that there's something hanging somewhere...
    for batch_no, (batch, labels) in enumerate(val_loader):
        
        # clean up garbage and clear cuda cache as much as possible 
        gc.collect()
        print "BATCH NUMBER: %s" % batch_no
        assert sorted(vars().keys()) == sorted(['labels', 'val_loader', 'batch', 'batch_no'])
        torch.cuda.empty_cache()
        
        
        # load things needed for attack 
        base_model = cifar_resnets.resnet32()
        adv_trained_net = checkpoints.load_state_dict_from_filename('half_trained_madry.th', base_model)
        #adv_trained_net.cuda()
        cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                           std=config.CIFAR10_STDS)        
        pgd_perceptual_loss = plf.PerceptualXentropy(adv_trained_net, normalizer=cifar_normer, use_gpu=True)            
        pgd_attack_obj = aa.LInfPGD(adv_trained_net, cifar_normer, pgd_perceptual_loss)
        
        adv_images = pgd_attack_obj.attack(batch, labels, l_inf_bound =8.0/255.0, 
                                           step_size=1.0/255.0, num_iterations=16, verbose=False)
        
        # push things to cpu (in hopes it gets them out of the cache)
        # also delete everything and be sure to collect garbage before next batch 
        batch.cpu()
        labels.cpu()
        del adv_images
        del batch 
        del labels 
        del pgd_attack_obj 
        del pgd_perceptual_loss
        del cifar_normer
        adv_trained_net.cpu()
        del adv_trained_net 
        del base_model 
    return
    
    
##########################################################################
#   BREAK THE PLANET BLOCK                                               #
##########################################################################
print memout_example()
print "SOMEHOW THIS WORKED??"

Files already downloaded and verified
BATCH NUMBER: 0


RuntimeError: Expected object of type Variable[torch.FloatTensor] but found type Variable[torch.cuda.FloatTensor] for argument #1 'other'